In [1]:
import torch
#from denoising_diffusion_pytorch import Unet, GaussianDiffusion
#from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D
from denoising_diffusion_pytorch import *
import numpy as np
import matplotlib.pyplot as plt
#from scipy.stats import skew,kurtosis
from PIL import Image

In [3]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [ ]:
pip install torch torchvision numpy matplotlib einops ema_pytorch accelerate pytorch_fid

## Load the data

In [8]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/data_prepared/data_norm_bhat/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [9]:
data = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [10]:
# (40, 40, 99) (videonumber/trialnumber , channelnumber, datapoints)
data['sub01'].shape

(40, 40, 99)

In [11]:
# matrix 1
data['sub01'][0].shape

(40, 99)

In [12]:
matrix = np.zeros((880,40,104))
matrix[:] = np.nan
counter = 0
for sub in data.keys():
    if sub == 'sub23':
        break
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        matrix[counter, :] = temp
        counter = counter + 1

In [13]:
matrix.shape

(880, 40, 104)

In [14]:
!mkdir images
for i in range(880):
    #transpose = np.transpose(matrix[i,:,:], (1,2,0))
    tra = np.multiply(matrix[i,:,:], 255).astype(np.uint8)
    img = Image.fromarray(tra)
    img.save("images2/train"+str(i)+".png")

------

### PNG and JPEG comparison

In [ ]:
test = cv2.imread('train20.jpeg')
print(test.shape)
test = np.transpose(test, (2,0,1))
test = np.multiply(test,1/255)
test

In [ ]:
test2 = cv2.imread('train20.png')
print(test.shape)
test2 = np.transpose(test2, (2,0,1))
test2 = np.multiply(test2,1/255)
test2

In [ ]:
np.allclose(fo, test2, rtol=0.1, atol=0.1)

For two images the png format was closer to the original input with a tolerance of 0.1 which failed in the case of jpeg. Therefore, png will be used to store the images

----

In [41]:
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    channels = 1
)

diffusion = GaussianDiffusion(
    model,
    image_size = (40,104),
    timesteps = 1000,           # number of steps
    sampling_timesteps = 250,   # number of sampling timesteps (using ddim for faster inference [see citation for ddim paper])
    loss_type = 'l2'            # L1 or L2
)

trainer = Trainer(
    diffusion,
    'images/',
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 12000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = False,                       # turn on mixed precision
    calculate_fid = True              # whether to calculate fid during training
)
#trainer.load('5')
trainer.train()

loading from version 1.5.6


In [43]:
for i in range(10):
    sampled_images = diffusion.sample(batch_size = 128)
    torch.save(sampled_images,'sampled1D_2000_128_'+str(i))
    print("Finished Sample: ",i)

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

Finished Sample:  0


sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

Finished Sample:  1


sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

...

Finished Sample:  7


sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

Finished Sample:  8


sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

Finished Sample:  9
